# BM25

In [2]:
import pandas as pd
from funcs import load_datasets, get_docs

In [3]:
datasets = load_datasets(["ru", "zh", "fa"])

In [4]:
documents = pd.DataFrame(datasets["ru"].docs_iter())

documents.count()

doc_id     964719
title      964719
text       964719
url        964719
time       215421
cc_file    964719
dtype: int64

In [5]:
print(documents[:5])

                                 doc_id  \
0  ecd810c8-4b67-4a53-a0bb-20e0214becde   
1  bdcf1b07-7d19-41a8-923d-55d08957a8d6   
2  b148f67a-8605-48d9-b032-f32a2280f1f0   
3  fcd39864-6cf5-4193-8903-9a101b6863ba   
4  2a0acf64-5fd4-43af-acbf-3f728d65ca2a   

                                               title  \
0  Рафаэль Надаль – в четвертьфинале Открытого че...   
1  Житель Октябрьского района, обналичив чужую ка...   
2  Воспоминания участников войны в Афганистане из...   
3    Глава спецслужбы ФРГ Масен отправлен в отставку   
4  Европейские индексы - 02-04-18 | Новости Армен...   

                                                text  \
0  Двое друзей встретились в парке, гуляя с собак...   
1  Вы нашли ошибку\n\nКакой то текст который мы в...   
2  Бежит мышка от кота, прыгает со стола и попада...   
3  Председатель Федерального ведомства по охране ...   
4  МОСКВА, 3 АПРЕЛЯ, АРМЕНПРЕСС. Результаты торго...   

                                                 url  \
0  https://

In [24]:
queries = pd.DataFrame(datasets["ru"].queries_iter())

queries

,query_id,title,description,ht_title,ht_description,mt_title,mt_description,narrative_by_relevance,report,report_url,report_date,translation_lang
0,3,British royal news impacts,What political and economic impacts does news ...,Влияние британских королевских новостей,Какое политическое и экономическое влияние нов...,Британские королевские новости влияют,Какие политические и экономические последствия...,{'very_valuable': 'Information regarding econo...,Announcement of engagement\nPrincess Eugenie o...,https://en.wikipedia.org/w/index.php?title=Wed...,2018-05-13,ru
1,6,Gibraltar's Sovereignty After Brexit,How will Gibraltar's sovereignty be impacted b...,Суверенитет Гибралтара после Брексита,Как на суверенитет Гибралтара повлияет на пере...,Суверенитет Гибралтара после Брексита,Как повлияют на суверенитет Гибралтара перегов...,{'very_valuable': 'Gibraltar will remain a ter...,Sovereignty\nSee also: Disputed status of Gibr...,https://en.wikipedia.org/w/index.php?title=Eff...,2018-11-26,ru
2,13,US-South Korea Trade Agreement,How will South Korea benefit from or be harmed...,США-Южнокорейское торговое соглашение,Как Южная Корея выиграет или пострадает от пер...,Торговое соглашение между США и Южной Кореей,Каким образом Южная Корея извлечет выгоду из с...,"{'very_valuable': 'N/A', 'somewhat_valuable': ...",South Korea reactions\n\t\nThis section is in ...,https://en.wikipedia.org/w/index.php?title=Uni...,2018-02-20,ru
3,14,North Korean Earthquakes and Nuclear Testing,Are earthquakes in North Korea being caused by...,Северокорейские землетрясения и ядерное испытание,Являются ли землетрясения в Северной Корее сле...,Северокорейские землетрясения и ядерные испытания,Являются ли землетрясения в Северной Корее при...,"{'very_valuable': 'N/A', 'somewhat_valuable': ...","On 3 September, at 3:31 AM UTC, the United Sta...",https://en.wikipedia.org/w/index.php?title=201...,2017-09-22,ru
4,101,Shipwrecks and Historical European Trade,What information about trade and shipping has ...,Кораблекрушения и историческая Европейская тор...,Какая информация о торговле и судоходстве была...,Кораблекрушения и историческая европейская тор...,Какая информация о торговле и судоходстве была...,{'very_valuable': 'Shipwrecks that belonged to...,Portuguese discoveries (Portuguese: Descobrime...,https://en.wikipedia.org/w/index.php?title=Por...,2018-05-16,ru
5,103,African Extremist School Girls Kidnapping,What is known about instances of extremist or ...,Похищение школьниц из числа африканских экстре...,"Что известно о случаях, когда экстремистские и...",Африканские Экстремистские Школьные Девушки По...,Что известно о случаях похищения девочками из ...,{'very_valuable': 'The number and instances of...,"On the night of 14–15 April 2014, 276 female s...",https://en.wikipedia.org/w/index.php?title=Chi...,2018-02-11,ru
6,105,Mount Sinabung Eruptions,How long will Mount Sinabung's eruptions last ...,Извержения горы Синабунг,Как долго продлится извержение горы Синабунг и...,Разрушения на горе Синабунг,Как долго будут продолжаться извержения горы С...,"{'very_valuable': 'None found', 'somewhat_valu...",Mount Sinabung erupted after a centuries-long ...,https://en.wikipedia.org/w/index.php?title=Mou...,2018-02-19,ru
7,107,UN Climate Change Economic Analysis,What is the economic impact of climate change ...,ООН Изменение Климата Экономический Анализ,Какие экономические последствия изменения клим...,Экономический анализ изменения климата ООН,Каково экономическое воздействие изменения кли...,{'very_valuable': 'Articles that described the...,The United Nations Framework Convention on Cli...,https://en.wikipedia.org/w/index.php?title=Uni...,2018-10-06,ru
8,108,Saudi Arabic movie theater restrictions,What restrictions did Saudi Arabia put on movi...,Ограничения кинотеатра в Саудовской Аравии,Какие ограничения Саудовская Аравия наложила н...,Ограничения саудовского арабского кинотеатра,Какие ограничения были введены Саудовской Арав...,{'very_valuable': 'Described the restriction

In [6]:
queries.to_excel("translated_queries.xlsx", index=False)

In [ ]:
# Load translated queries from the Excel file
translated_queries = pd.read_excel("translated_queries.xlsx")

In [39]:
# Verify the processed queries
print(translated_queries[:5])

   query_id                                         title  \
0         3                    British royal news impacts   
1         6          Gibraltar's Sovereignty After Brexit   
2        13                US-South Korea Trade Agreement   
3        14  North Korean Earthquakes and Nuclear Testing   
4       101      Shipwrecks and Historical European Trade   

                                         description  \
0  What political and economic impacts does news ...   
1  How will Gibraltar's sovereignty be impacted b...   
2  How will South Korea benefit from or be harmed...   
3  Are earthquakes in North Korea being caused by...   
4  What information about trade and shipping has ...   

                                    ht_title  \
0               Impact of British Royal News   
1      Sovereignty of Gibraltar after Brexit   
2             US-South Korea Trade Agreement   
3  North Korean earthquakes and nuclear test   
4   Shipwrecks and historical European trade   

       

In [7]:
#tokenize and normalize

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

#nltk.download('punkt')
#nltk.download('stopwords')

def preprocess(text):
    # Tokenize
    tokens = word_tokenize(text)
    # Lowercase and remove punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)  # Join tokens back into a single string

# Apply preprocessing to the document text
documents['processed_text'] = documents['text'].apply(preprocess)


In [ ]:
documents.to_excel("processed_documents.xlsx", index=False)

documents = pd.read_excel("processed_documents.xlsx")

In [9]:
# Convert the processed documents to a list
processed_documents = documents["processed_text"].tolist()

# Verify a few processed documents
print(processed_documents[:5])

['двое друзей встретились в парке гуляя с собаками один из них предложил зайти позавтракать в ближайшее кафе нас же не пустят туда с собаками возразил второй первый решительно направился в кафе со своей немецкой овчаркой но хозяин остановил его со словами сэр к нам нельзя заходить с животными но я слепой а это моя хозяин извинился и проводил его с собакой к столику его друг подождал на улице пять минут а потом попробовал сделать то же самое ваш поводырь чихуахуа скептически осведомился хозяин чихуахуа удивился мужчина так вот что они мне подсунули еще анекдот', 'вы нашли ошибку какой то текст который мы выделяем и смотрим его тут вы выделили слишком много максимальное количество символов попробуйте снова спасибо сообщение отправлено мы скоро все исправим', 'бежит мышка от кота прыгает со стола и попадает в бутылку с недопитым вином стоящую на полу барахтается она там и говорит коту вытащи не дай умереть такой кошмарной смертью а не убежишь нет честное слово нет тот ее вытащил а она пер

In [10]:
# Apply preprocessing to the 'title' column
translated_queries["processed_query"] = translated_queries["title"].apply(preprocess)

# Convert the processed queries to a list
processed_queries = translated_queries["processed_query"].tolist()

# Verify the processed queries
print(processed_queries[:5])

['british royal news impacts', 'gibraltar sovereignty brexit', 'korea trade agreement', 'north korean earthquakes nuclear testing', 'shipwrecks historical european trade']


In [11]:
from rank_bm25 import BM25Okapi


In [12]:
# Initialize the BM25 model with the tokenized corpus
bm25 = BM25Okapi(processed_documents)

In [13]:
# Example query: Retrieve top documents for each query
results = {}
for idx, query in translated_queries.iterrows():
    query_tokens = query['processed_query']
    doc_scores = bm25.get_scores(query_tokens)
    top_n_indices = doc_scores.argsort()[-10:][::-1]  # Top 10 documents
    results[query['query_id']] = documents.iloc[top_n_indices][['doc_id', 'title', 'text']]


In [14]:
# View results for a specific query_id
query_id_to_view = translated_queries['query_id'].iloc[0]
print(f"Top documents for query {query_id_to_view}:")
print(results[query_id_to_view])

Top documents for query 3:
                                      doc_id  \
275620  282daef5-dbf5-4d89-96b0-0cefb06408e2   
45255   db368871-2a5b-45ec-9540-7dc4557744ff   
314647  78273e10-8ca6-4a38-9dad-d7ecb89fdadc   
545603  9d3f9617-e723-4a1f-80e1-3c357d1f9347   
338842  81d13a60-eb73-48a0-91b7-f0460d4f6ffe   
683789  e3492b0e-47ed-4237-b268-fd208453ad0e   
743999  ce191847-f9c3-47ed-a355-476ebffa3602   
945829  5ef1d191-7afa-4c3e-8910-053505f498e5   
62332   2b7e0c2d-7ef5-4408-802b-4d3c042bf43a   
12093   b1b5c1e5-b65a-40d1-b0b7-06bd75e2fcce   

                                                    title  \
275620                          Тур Лангкави-2018. Этап 7   
45255                              Тур Омана-2017. Этап 2   
314647       Ящик пандоры – Олигархов нет, а санкции есть   
545603                      Tour de l’Avenir-2018. Этап 7   
338842                          Тур Хорватии-2018. Этап 4   
683789  Список номинантов игровой премии GDC Awards 20...   
743999           

# Evaluation

In [15]:
bm25_run = []

# Convert BM25 results into the required format
for query_id, docs in results.items():
    # Compute BM25 scores for the query
    query_tokens = translated_queries.loc[translated_queries["query_id"] == query_id, "processed_query"].values[0]
    doc_scores = bm25.get_scores(query_tokens)
    
    # Iterate over the retrieved documents
    for doc_index in docs.index:
        doc_id = docs.at[doc_index, "doc_id"]
        score = doc_scores[documents.index[documents["doc_id"] == doc_id][0]]  # Map the score by index
        bm25_run.append({'query_id': query_id, 'doc_id': doc_id, 'score': score})

your_run = pd.DataFrame(bm25_run)

print("BM25 Run DataFrame:\n", your_run.head())

BM25 Run DataFrame:
    query_id                                doc_id       score
0         3  282daef5-dbf5-4d89-96b0-0cefb06408e2  144.299030
1         3  db368871-2a5b-45ec-9540-7dc4557744ff  144.119638
2         3  78273e10-8ca6-4a38-9dad-d7ecb89fdadc  143.928913
3         3  9d3f9617-e723-4a1f-80e1-3c357d1f9347  143.775420
4         3  81d13a60-eb73-48a0-91b7-f0460d4f6ffe  143.381528


In [16]:
# Load qrels from dataset
qrels = pd.DataFrame(datasets["ru"].qrels_iter())

print("Loaded qrels:\n", qrels.head())


Loaded qrels:
   query_id                                doc_id  relevance iteration
0        3  07d5796e-e55f-469f-b158-88a8aa53131d          0         0
1        3  084c6487-5838-4043-98e3-c3837bf6e484          1         0
2        3  0997514d-dc9c-478b-bb0f-abb9747f2b11          0         0
3        3  12239503-c6a6-40cb-9bb6-3452fa9b3b99          0         0
4        3  158e7a45-76ee-4746-a2d0-ca97880476b1          0         0


In [17]:
# Check the count of each relevance score in qrels
relevance_counts = qrels["relevance"].value_counts()

In [18]:
# Determine the minimum count among relevance scores to ensure balanced sampling
min_sample_count = 0

In [19]:
# Sample equally from each relevance score
sample_0 = qrels[qrels["relevance"] == 0].sample(n=min_sample_count, random_state=42)
sample_1 = qrels[qrels["relevance"] == 1].sample(n=min_sample_count, random_state=42)
sample_3 = qrels[qrels["relevance"] == 3].sample(n=min_sample_count, random_state=42)

In [20]:
sample = pd.concat([sample_0, sample_1, sample_3]).reset_index(drop=True)

In [21]:
use_full_dataset = input("Run Training on Full HC4 RU Dataset? (y/n)")

if use_full_dataset == "y":
    sample = qrels

# Enables flexibility in choosing the evaluation dataset.

In [25]:
doc_ids = sample["doc_id"].values
query_ids = sample["query_id"].values

docs = documents[documents["doc_id"].isin(doc_ids)]
queries = queries[queries["query_id"].isin(query_ids)]

# Subset documents and queries to include only those that are in the sampled qrels.

In [26]:
# Find overlapping query_ids
common_query_ids = set(qrels["query_id"]).intersection(your_run["query_id"])

# Filter qrels and your_run to keep only common query_ids
qrels_filtered = qrels[qrels["query_id"].isin(common_query_ids)]
your_run_filtered = your_run[your_run["query_id"].isin(common_query_ids)]

# Ensure document IDs also overlap
common_doc_ids = set(qrels_filtered["doc_id"]).intersection(your_run_filtered["doc_id"])
qrels_filtered = qrels_filtered[qrels_filtered["doc_id"].isin(common_doc_ids)]
your_run_filtered = your_run_filtered[your_run_filtered["doc_id"].isin(common_doc_ids)]

In [28]:
# Check overlap in query_ids
print("Query IDs in qrels:", set(qrels["query_id"]))
print("Query IDs in your_run:", set(your_run["query_id"]))
print("Common Query IDs:", set(qrels["query_id"]).intersection(your_run["query_id"]))

# Check overlap in doc_ids
print("Doc IDs in qrels:", set(qrels["doc_id"]))
print("Doc IDs in your_run:", set(your_run["doc_id"]))
print("Common Doc IDs:", set(qrels["doc_id"]).intersection(your_run["doc_id"]))


Query IDs in qrels: {'247', '232', '199', '6', '142', '113', '114', '164', '185', '246', '248', '137', '136', '135', '101', '128', '150', '14', '158', '253', '111', '103', '146', '105', '255', '245', '13', '126', '157', '116', '138', '179', '133', '251', '3', '252', '208', '108', '234', '192', '230', '127', '233', '107', '229', '249', '256', '254', '161', '250', '134', '151', '172', '231'}
Query IDs in your_run: {128, 256, 3, 255, 133, 6, 134, 135, 136, 137, 138, 13, 14, 142, 146, 150, 151, 157, 158, 161, 164, 172, 179, 185, 192, 199, 208, 254, 101, 229, 103, 230, 105, 231, 107, 108, 232, 233, 111, 234, 113, 114, 116, 245, 246, 247, 248, 249, 250, 251, 252, 253, 126, 127}
Common Query IDs: set()
Doc IDs in qrels: {'78275722-8275-4f9e-a657-80635c53d477', '977f3409-f713-47d8-91c6-12168fe07652', '80b1fc86-74de-470c-a0e7-d6f50f56b3b9', '85ba0b76-1eb6-4722-98be-566ba734d200', '1ae4d69c-c955-4256-87c2-10aeb47e27ee', 'a57648bf-de2c-4823-8f41-ede90778c887', '80e0bdd5-a280-4679-aa8b-20b3e3bf145

In [29]:
# Check Column Names and Types

qrels["query_id"] = qrels["query_id"].astype(str)
qrels["doc_id"] = qrels["doc_id"].astype(str)
your_run["query_id"] = your_run["query_id"].astype(str)
your_run["doc_id"] = your_run["doc_id"].astype(str)

In [35]:
# Filter qrels to only include query IDs and doc IDs present in your_run
filtered_qrels = qrels[
    qrels["query_id"].isin(your_run["query_id"]) &
    qrels["doc_id"].isin(your_run["doc_id"])
]

# Filter your_run to only include query IDs and doc IDs present in qrels
filtered_run = your_run[
    your_run["query_id"].isin(qrels["query_id"]) &
    your_run["doc_id"].isin(qrels["doc_id"])
]

print("Filtered qrels:\n", filtered_qrels)
print("Filtered your_run:\n", filtered_run)


Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFrame
Columns: [query_id, doc_id, score]
Index: []


In [37]:
import ir_measures
from ir_measures import nDCG, P, Judged

evaluation_metrics = ir_measures.calc_aggregate(
    [nDCG@10, P@5, P(rel=2)@5, Judged@10],
    filtered_qrels,
    filtered_run
)

print("BM25 Evaluation Metrics:\n", evaluation_metrics)


BM25 Evaluation Metrics:
 {P@5: nan, Judged@10: nan, nDCG@10: nan, P(rel=2)@5: nan}


In [33]:
# Sample doc_ids from qrels and your_run
print("Sample qrels doc_ids:\n", qrels["doc_id"].head())
print("Sample your_run doc_ids:\n", your_run["doc_id"].head())

# Check for common patterns
print("qrels doc_id example:", qrels["doc_id"].iloc[0])
print("your_run doc_id example:", your_run["doc_id"].iloc[0])


Sample qrels doc_ids:
 0    07d5796e-e55f-469f-b158-88a8aa53131d
1    084c6487-5838-4043-98e3-c3837bf6e484
2    0997514d-dc9c-478b-bb0f-abb9747f2b11
3    12239503-c6a6-40cb-9bb6-3452fa9b3b99
4    158e7a45-76ee-4746-a2d0-ca97880476b1
Name: doc_id, dtype: object
Sample your_run doc_ids:
 0    282daef5-dbf5-4d89-96b0-0cefb06408e2
1    db368871-2a5b-45ec-9540-7dc4557744ff
2    78273e10-8ca6-4a38-9dad-d7ecb89fdadc
3    9d3f9617-e723-4a1f-80e1-3c357d1f9347
4    81d13a60-eb73-48a0-91b7-f0460d4f6ffe
Name: doc_id, dtype: object
qrels doc_id example: 07d5796e-e55f-469f-b158-88a8aa53131d
your_run doc_id example: 282daef5-dbf5-4d89-96b0-0cefb06408e2


In [61]:
# Normalize doc_ids
qrels["doc_id"] = qrels["doc_id"].str.strip().str.lower()
your_run["doc_id"] = your_run["doc_id"].str.strip().str.lower()

# Check common doc_ids again
common_doc_ids = set(qrels["doc_id"]) & set(your_run["doc_id"])
print("Common document IDs after normalization:", common_doc_ids)


Common document IDs after normalization: set()


In [ ]:
# Verify existence of specific doc_ids
sample_doc_id = qrels["doc_id"].iloc[0]
print(f"Sample doc_id {sample_doc_id} exists in your_run:", sample_doc_id in your_run["doc_id"].values)

Sample doc_id 07d5796e-e55f-469f-b158-88a8aa53131d exists in your_run: False


In [80]:
print("BM25 scores for query:", bm25.get_scores(query_tokens))


BM25 scores for query: [18.73885141 18.55341715 18.72660017 ... 18.73590567 18.74199749
 18.78944082]


# Now with parameter tuning

In [38]:
from rank_bm25 import BM25Okapi
import pandas as pd
import numpy as np
import ir_measures
from ir_measures import nDCG, P, Judged

# Parameter ranges for tuning
k1_values = [1.2, 1.5, 1.8, 2.0]
b_values = [0.5, 0.75, 1.0]

# Initialize variables to track the best parameters
best_params = None
best_score = 0
results_with_params = []

# Iterate over different k1 and b combinations
for k1 in k1_values:
    for b in b_values:
        print(f"Testing BM25 with k1={k1}, b={b}")
        # Initialize BM25 with current parameters
        bm25 = BM25Okapi(processed_documents, k1=k1, b=b)

        # Prepare BM25 results for all queries
        bm25_run = []
        for idx, query in translated_queries.iterrows():
            query_id = query['query_id']
            query_tokens = query['processed_query']

            # Compute scores for all documents
            doc_scores = bm25.get_scores(query_tokens)

            # Retrieve the top 10 documents for this query
            top_n_indices = doc_scores.argsort()[-10:][::-1]
            for doc_index in top_n_indices:
                doc_id = documents.iloc[doc_index]['doc_id']
                score = doc_scores[doc_index]
                bm25_run.append({'query_id': query_id, 'doc_id': doc_id, 'score': score})

        # Convert results into a DataFrame
        your_run = pd.DataFrame(bm25_run)

        # Filter qrels to only include query IDs and doc IDs present in your_run
        filtered_qrels = qrels[
            qrels["query_id"].isin(your_run["query_id"]) &
            qrels["doc_id"].isin(your_run["doc_id"])
        ]

        # Filter your_run to only include query IDs and doc IDs present in qrels
        filtered_your_run = your_run[
            your_run["query_id"].isin(qrels["query_id"]) &
            your_run["doc_id"].isin(qrels["doc_id"])
        ]

        print("Filtered qrels:\n", filtered_qrels.head())
        print("Filtered your_run:\n", filtered_your_run.head())

        # Evaluate using IR measures
        try:
            evaluation_metrics = ir_measures.calc_aggregate(
                [nDCG@10, P@5, P(rel=2)@5, Judged@10],
                filtered_qrels,
                filtered_your_run
            )
        except Exception as e:
            print(f"Error calculating IR measures for k1={k1}, b={b}: {e}")
            continue

        # Store metrics and parameters
        results_with_params.append({'k1': k1, 'b': b, 'metrics': evaluation_metrics})
        print(f"Metrics for k1={k1}, b={b}: {evaluation_metrics}")

        # Update the best score and parameters if applicable
        if evaluation_metrics[nDCG@10] > best_score:
            best_score = evaluation_metrics[nDCG@10]
            best_params = (k1, b)

# Display the best parameters and their performance
print("\nBest parameters:")
print(f"k1={best_params[0]}, b={best_params[1]}, nDCG@10={best_score}")

# Optional: Save all results to a DataFrame
results_df = pd.DataFrame(results_with_params)
print("\nAll parameter tuning results:")
print(results_df)


Testing BM25 with k1=1.2, b=0.5
Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFrame
Columns: [query_id, doc_id, score]
Index: []
Metrics for k1=1.2, b=0.5: {P@5: nan, Judged@10: nan, nDCG@10: nan, P(rel=2)@5: nan}
Testing BM25 with k1=1.2, b=0.75
Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFrame
Columns: [query_id, doc_id, score]
Index: []
Metrics for k1=1.2, b=0.75: {P@5: nan, Judged@10: nan, nDCG@10: nan, P(rel=2)@5: nan}
Testing BM25 with k1=1.2, b=1.0


c:\Users\nyash\AppData\Local\Programs\Python\Python312\Lib\site-packages\rank_bm25.py:119: RuntimeWarning: invalid value encountered in divide
  score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /


Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFrame
Columns: [query_id, doc_id, score]
Index: []
Metrics for k1=1.2, b=1.0: {P@5: nan, Judged@10: nan, nDCG@10: nan, P(rel=2)@5: nan}
Testing BM25 with k1=1.5, b=0.5
Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFrame
Columns: [query_id, doc_id, score]
Index: []
Metrics for k1=1.5, b=0.5: {P@5: nan, Judged@10: nan, nDCG@10: nan, P(rel=2)@5: nan}
Testing BM25 with k1=1.5, b=0.75
Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFrame
Columns: [query_id, doc_id, score]
Index: []
Metrics for k1=1.5, b=0.75: {P@5: nan, Judged@10: nan, nDCG@10: nan, P(rel=2)@5: nan}
Testing BM25 with k1=1.5, b=1.0
Filtered qrels:
 Empty DataFrame
Columns: [query_id, doc_id, relevance, iteration]
Index: []
Filtered your_run:
 Empty DataFr

KeyboardInterrupt: 